In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import networkx as nx
import seaborn as sns
import math

# Import data
*   Clean export data
*   Country list
*   Product list
*   GDP per Capita
*   ATLAS ECI rank



In [2]:
df = pd.read_csv('/content/drive/My Drive/data/PS_2018/nodup.csv')
#df

**Country list**

In [3]:
cty_list = pd.read_csv('/content/drive/MyDrive/data/PS_2018/country_list_3.csv')
cty_list

,Continent_Name,Country_Name,Three_Letter
0,Asia,"Afghanistan, Islamic Republic of",AFG
1,Europe,"Albania, Republic of",ALB
2,Antarctica,Antarctica (the territory South of 60 deg S),ATA
3,Africa,"Algeria, People's Democratic Republic of",DZA
4,Oceania,American Samoa,ASM
...,...,...,...
257,Africa,"Zambia, Republic of",ZMB
258,Oceania,Disputed Territory,NaN
259,Asia,Iraq-Saudi Arabia Neutral Zone,NaN
260,Asia,United Nations Neutral Zone,NaN


In [4]:
cty_list.rename({"Three_Letter":"country"},axis=1,inplace=True)

**Product code**

In [5]:
SITC2 = pd.read_csv('/content/drive/MyDrive/data/PS_2018/sitc2.csv')
#SITC2

In [6]:
SITC2['Commodity Code'] = SITC2['Commodity Code'].astype(str)

In [7]:
SITC2['Commodity Code'] = SITC2['Commodity Code'].str.zfill(2)
SITC2.rename({'Commodity Code':'two'}, axis=1, inplace=True)
SITC2

,two,Commodity description
0,00,Live animals chiefly for food
1,01,Meat and preparations
2,02,Dairy products and birds' eggs
3,03,"Fish, crustacean and molluscs, and preparation..."
4,04,Cereals and cereal preparations
...,...,...
64,93,"Special transactions, commodity not classified..."
65,94,"Animals, live, nes, (including zoo animals, pe..."
66,95,"Armoured fighting vehicles, war firearms, ammu..."
67,96,"Coin (other than gold coin), not being legal t..."


**GDP per Capita**

In [8]:
gdppercap=pd.read_excel('/content/drive/My Drive/data/PS_2018/gdppercap.xlsx')
gdppercap.columns = ['country','gdp']

**ATLAS ECI ranking**

In [9]:
#atlas = pd.read_csv('/content/drive/My Drive/data/PS_2018/ECI.csv')
#atlas

#Readjust data to separate the code with products names

In [10]:
df['sitc'] = df['sitc'].astype(str)
df.drop(columns =["Unnamed: 0"], inplace = True)
df.drop(columns =["cntry_export"], inplace = True)
df.drop(columns =["prod_export"], inplace = True)
df.drop(columns =["world_export"], inplace = True)
df.drop(columns =["export_value"], inplace = True)
df.rename(columns = {'location_code': 'country', 'sitc_product_code': 'sitc', 'dup': 'export'}, inplace= True)


In [11]:
df['sitc'] = df['sitc'].str.zfill(4)
df

,year,country,sitc,export
0,2018.0,ABW,0240,7.728600e+04
1,2018.0,ARE,0240,3.589986e+07
2,2018.0,AUT,0240,6.825887e+08
3,2018.0,DEU,0240,4.490879e+09
4,2018.0,DNK,0240,1.623338e+09
...,...,...,...,...
149393,2018.0,FRA,2860,0.000000e+00
149394,2018.0,IRL,7521,1.582000e+03
149395,2018.0,TTO,7521,0.000000e+00
149396,2018.0,USA,7521,7.305000e+04


In [12]:
temp = pd.pivot_table(df,values='export',index='country',columns='sitc').fillna(0)
temp

sitc,0011,0012,0013,0014,0015,0111,0112,0113,0114,0115,0116,0118,0121,0129,0141,0142,0149,0223,0224,0230,0240,0251,0252,0341,0342,0343,0344,0350,0360,0371,0372,0411,0412,0421,0422,0430,0440,0451,0452,0459,...,8841,8842,8851,8852,8921,8922,8924,8928,8931,8932,8933,8935,8939,8941,8942,8946,8947,8951,8952,8959,8960,8972,8973,8974,8981,8982,8983,8989,8991,8993,8994,8996,8997,8998,8999,9310,9410,9510,9610,9710
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ABW,0.0,0.0,0.0,74109.0,23826.0,2449.0,0.0,0.0,74276.0,0.0,0.0,0.0,32860.0,0.0,0.0,0.0,32386.0,34878.0,10581.0,84343.0,77286.0,0.0,0.0,2031.0,92682.0,0.0,0.0,85989.0,30333.0,98419.0,18295.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3879.0,475073.0,1587271.0,3932.0,34613.0,1067.0,3149.0,88780.0,458604.0,0.0,0.0,0.0,117582.0,0.0,183724.0,12005.0,9435.0,0.0,3413.0,2115.0,33593.0,61308.0,538786.0,0.0,2808.0,0.0,21523.0,0.0,0.0,0.0,4743.0,2385127.0,0.0,0.0,0.0,2.116981e+07,10626.0,6499.0,0.0,9.071140e+05
AFG,0.0,2344033.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3201.0,0.0,0.0,0.0,0.0,0.0,0.0,44138.0,5588.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4151.0,0.0,1689.0,0.0,3195.0,0.0,0.0,465990.0,0.0,0.0,0.0,385095.0,...,20884.0,0.0,2867.0,0.0,17003.0,0.0,0.0,20117.0,4307313.0,0.0,0.0,0.0,413322.0,0.0,28447.0,99764.0,71709.0,0.0,0.0,0.0,2626981.0,60347.0,249976.0,0.0,1187.0,0.0,4283.0,0.0,0.0,0.0,0.0,4506.0,10547.0,10558.0,3472.0,8.623021e+07,112859.0,2819.0,8374.0,7.980523e+07
AGO,13646.0,0.0,0.0,9499.0,0.0,30850.0,0.0,51049.0,45269.0,0.0,0.0,0.0,0.0,0.0,0.0,25398.0,92132.0,10071.0,288016.0,24413.0,4386.0,2234.0,3896.0,35175648.0,71489328.0,0.0,190790.0,2449315.0,29807905.0,32372.0,1360437.0,0.0,0.0,194893.0,173697.0,0.0,7367.0,0.0,0.0,2078.0,...,407461.0,43110.0,979479.0,0.0,50754.0,1791.0,0.0,214398.0,364029.0,126940.0,3753.0,0.0,2932284.0,0.0,263526.0,2708.0,328476.0,0.0,11318.0,7960.0,72216.0,26706.0,52195.0,0.0,6154.0,4474.0,82991.0,0.0,10736.0,73353.0,10366.0,4301.0,16566.0,12432.0,187122.0,3.857242e+07,3136340.0,0.0,0.0,2.883109e+06
AIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2891.0,0.0,0.0,0.0,...,28451.0,0.0,0.0,0.0,1616.0,0.0,4613.0,1505.0,0.0,0.0,0.0,0.0,79597.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1298397.0,5641.0,0.0,0.0,0.0,0.0,182302.0,0.0,0.0,0.0,0.0,49929.0,0.0,0.0,0.0,1.629800e+05,0.0,0.0,0.0,3.317000e+04
ALB,0.0,24660.0,0.0,0.0,0.0,10431.0,0.0,15748.0,0.0,337369.0,0.0,2267917.0,27395.0,0.0,0.0,14021.0,4454.0,1062.0,5269.0,0.0,79600.0,745407.0,22122.0,9837964.0,2099415.0,138822.0,996477.0,14832691.0,8134660.0,25180695.0,25243609.0,1022.0,0.0,0.0,9634.0,0.0,4754.0,0.0,0.0,0.0,...,48422.0,2338626.0,57019.0,0.0,566874.0,2258.0,8930.0,2318195.0,3138854.0,117208.0,876732.0,0.0,7686596.0,0.0,3582187.0,68807.0,225928.0,0.0,4465.0,28789.0,927071.0,238115.0,2872490.0,0.0,0.0,8093.0,63198.0,0.0,0.0,781937.0,17884.0,225774.0,1488424.0,84587.0,37966.0,5.263699e+08,1037572.0,0.0,0.0,7.785660e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.0,0.0,0.0,0.0,0.0,46277.0,26620.0,3216.0,1534095.0,0.0,219369.0,34476.0,0.0,24224.0,0.0,18423.0,113251.0,98724.0,25816.0,13740.0,23183.0,10103.0,6604.0,573067.0,12308928.0,255451.0,0.0,0.0,2489.0,39332.0,0.0,0.0,0.0,0.0,53819.0,0.0,0.0,0.0,1732.0,0.0,...,2772.0,6927.0,0.0,0.0,13819.0,0.0,4022.0,31755.0,13705.0,0.0,0.0,0.0,796095.0,0.0,32690.0,0.0,161350.0,0.0,11465.0,2173.0,0.0,9130.0,1652.0,0.0,0.0,0.0,3149.0,0.0,0.0,3888.0,133950.0,46371.0,14666.0,6841.0,3796.0,4.464630e+05,0.0,0.0,0.0,0.000000e+00
YEM,0.0,514572.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176545.0,6631990.0,68017.0,0.0,0.0,0.0,53366209.0,18572914.0,32520.0,7

In [13]:
data = pd.DataFrame(temp.stack().rename_axis(['country','sitc']).rename('export').reset_index())
data

,country,sitc,export
0,ABW,0011,0.0
1,ABW,0012,0.0
2,ABW,0013,0.0
3,ABW,0014,74109.0
4,ABW,0015,23826.0
...,...,...,...
181007,ZWE,9310,103325888.0
181008,ZWE,9410,746033.0
181009,ZWE,9510,2241.0
181010,ZWE,9610,0.0


#Creating new variables for RCA

In [14]:
#Calculating part of RCA
data["cntry_export"] = data[["country", "export"]].groupby(by=["country"]).transform(np.sum)
data["prod_export"] = data[["sitc", "export"]].groupby(by=["sitc"]).transform(np.sum)
data["world_export"] = data['export'].sum(axis=0)
data

,country,sitc,export,cntry_export,prod_export,world_export
0,ABW,0011,0.0,2.128365e+08,9.295920e+09,1.875183e+13
1,ABW,0012,0.0,2.128365e+08,1.788313e+09,1.875183e+13
2,ABW,0013,0.0,2.128365e+08,3.763658e+09,1.875183e+13
3,ABW,0014,74109.0,2.128365e+08,3.197278e+09,1.875183e+13
4,ABW,0015,23826.0,2.128365e+08,2.954505e+09,1.875183e+13
...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,2.193997e+09,7.072683e+11,1.875183e+13
181008,ZWE,9410,746033.0,2.193997e+09,1.182526e+09,1.875183e+13
181009,ZWE,9510,2241.0,2.193997e+09,1.344083e+10,1.875183e+13
181010,ZWE,9610,0.0,2.193997e+09,4.515294e+08,1.875183e+13


In [15]:
#RCA
data["rca"] = (data["export"] / data["cntry_export"])/(data["prod_export"] / data["world_export"]).fillna(0)
data

,country,sitc,export,cntry_export,prod_export,world_export,rca
0,ABW,0011,0.0,2.128365e+08,9.295920e+09,1.875183e+13,0.000000
1,ABW,0012,0.0,2.128365e+08,1.788313e+09,1.875183e+13,0.000000
2,ABW,0013,0.0,2.128365e+08,3.763658e+09,1.875183e+13,0.000000
3,ABW,0014,74109.0,2.128365e+08,3.197278e+09,1.875183e+13,2.042153
4,ABW,0015,23826.0,2.128365e+08,2.954505e+09,1.875183e+13,0.710500
...,...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,2.193997e+09,7.072683e+11,1.875183e+13,1.248627
181008,ZWE,9410,746033.0,2.193997e+09,1.182526e+09,1.875183e+13,5.392063
181009,ZWE,9510,2241.0,2.193997e+09,1.344083e+10,1.875183e+13,0.001425
181010,ZWE,9610,0.0,2.193997e+09,4.515294e+08,1.875183e+13,0.000000


In [16]:
data.loc[data['sitc']=='0240']#look for specific product

,country,sitc,export,cntry_export,prod_export,world_export,rca
20,ABW,0240,77286.0,2.128365e+08,3.080253e+10,1.875183e+13,0.221061
787,AFG,0240,0.0,1.189510e+09,3.080253e+10,1.875183e+13,0.000000
1554,AGO,0240,4386.0,4.189613e+10,3.080253e+10,1.875183e+13,0.000064
2321,AIA,0240,0.0,1.210855e+07,3.080253e+10,1.875183e+13,0.000000
3088,ALB,0240,79600.0,2.836856e+09,3.080253e+10,1.875183e+13,0.017082
...,...,...,...,...,...,...,...
177197,WSM,0240,23183.0,4.768626e+07,3.080253e+10,1.875183e+13,0.295960
177964,YEM,0240,0.0,1.402799e+09,3.080253e+10,1.875183e+13,0.000000
178731,ZAF,0240,40974529.0,1.070877e+11,3.080253e+10,1.875183e+13,0.232933
179498,ZMB,0240,3538.0,8.348409e+09,3.080253e+10,1.875183e+13,0.000258


#Mcp Column

In [17]:
data["mcp"] = data["rca"].apply(lambda x: 1 if x >= 1.0 else 0.0)
data

,country,sitc,export,cntry_export,prod_export,world_export,rca,mcp
0,ABW,0011,0.0,2.128365e+08,9.295920e+09,1.875183e+13,0.000000,0.0
1,ABW,0012,0.0,2.128365e+08,1.788313e+09,1.875183e+13,0.000000,0.0
2,ABW,0013,0.0,2.128365e+08,3.763658e+09,1.875183e+13,0.000000,0.0
3,ABW,0014,74109.0,2.128365e+08,3.197278e+09,1.875183e+13,2.042153,1.0
4,ABW,0015,23826.0,2.128365e+08,2.954505e+09,1.875183e+13,0.710500,0.0
...,...,...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,2.193997e+09,7.072683e+11,1.875183e+13,1.248627,1.0
181008,ZWE,9410,746033.0,2.193997e+09,1.182526e+09,1.875183e+13,5.392063,1.0
181009,ZWE,9510,2241.0,2.193997e+09,1.344083e+10,1.875183e+13,0.001425,0.0
181010,ZWE,9610,0.0,2.193997e+09,4.515294e+08,1.875183e+13,0.000000,0.0


In [18]:
#MCP in country and product
mcp_table = pd.pivot_table(data,values='mcp',index='country',columns='sitc').fillna(0)
mcp_table

sitc,0011,0012,0013,0014,0015,0111,0112,0113,0114,0115,0116,0118,0121,0129,0141,0142,0149,0223,0224,0230,0240,0251,0252,0341,0342,0343,0344,0350,0360,0371,0372,0411,0412,0421,0422,0430,0440,0451,0452,0459,...,8841,8842,8851,8852,8921,8922,8924,8928,8931,8932,8933,8935,8939,8941,8942,8946,8947,8951,8952,8959,8960,8972,8973,8974,8981,8982,8983,8989,8991,8993,8994,8996,8997,8998,8999,9310,9410,9510,9610,9710
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ABW,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AFG,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
AGO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ALB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YEM,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ZAF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [19]:
#MCP for specific country
products = mcp_table.loc['KIR']
products[products==1]

sitc
0341    1.0
0342    1.0
0344    1.0
2231    1.0
2783    1.0
2911    1.0
4243    1.0
5834    1.0
7272    1.0
Name: KIR, dtype: float64

In [20]:
#validating diversity
len(products[products==1])

9

In [21]:
pd.set_option("max_rows", 10)

MCP with SITC

In [22]:
#display mcp in transpose
table1 = pd.pivot_table(data,values='mcp',index='sitc',columns='country').fillna(0)
table1

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
sitc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0012,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0014,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0015,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9310,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9410,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [23]:
#MCP for specific SITC CODE
sitcprod = table1.loc['3413']
sitcprod[sitcprod==1]

country
ABW    1.0
AGO    1.0
ARE    1.0
ASM    1.0
AUS    1.0
      ... 
SAU    1.0
SLV    1.0
TTO    1.0
USA    1.0
VCT    1.0
Name: 3413, Length: 37, dtype: float64

In [24]:
#validating Ubiquity
len(sitcprod[sitcprod==1])

37

In [25]:
#Diversity
data["diversity"] = data[["country", "mcp"]].groupby(by=["country"]).transform(np.sum)
#Ubiquity
data["ubiquity"] = data[["sitc", "mcp"]].groupby(by=["sitc"]).transform(np.sum)
data

,country,sitc,export,cntry_export,prod_export,world_export,rca,mcp,diversity,ubiquity
0,ABW,0011,0.0,2.128365e+08,9.295920e+09,1.875183e+13,0.000000,0.0,42.0,40.0
1,ABW,0012,0.0,2.128365e+08,1.788313e+09,1.875183e+13,0.000000,0.0,42.0,30.0
2,ABW,0013,0.0,2.128365e+08,3.763658e+09,1.875183e+13,0.000000,0.0,42.0,17.0
3,ABW,0014,74109.0,2.128365e+08,3.197278e+09,1.875183e+13,2.042153,1.0,42.0,36.0
4,ABW,0015,23826.0,2.128365e+08,2.954505e+09,1.875183e+13,0.710500,0.0,42.0,21.0
...,...,...,...,...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,2.193997e+09,7.072683e+11,1.875183e+13,1.248627,1.0,79.0,65.0
181008,ZWE,9410,746033.0,2.193997e+09,1.182526e+09,1.875183e+13,5.392063,1.0,79.0,61.0
181009,ZWE,9510,2241.0,2.193997e+09,1.344083e+10,1.875183e+13,0.001425,0.0,79.0,21.0
181010,ZWE,9610,0.0,2.193997e+09,4.515294e+08,1.875183e+13,0.000000,0.0,79.0,35.0


#Diversity and Ubiquity





In [26]:
  #DIVERSITY table
  divtable = pd.pivot_table(data,values='mcp',index='country',aggfunc= np.sum)
  #pd.set_option("max_rows", None)
  divtable

,mcp
country,
ABW,42.0
AFG,65.0
AGO,16.0
AIA,44.0
ALB,103.0
...,...
WSM,96.0
YEM,50.0
ZAF,158.0


In [27]:
#UBIQUITY table
ubitable = pd.pivot_table(data,values='mcp',index='sitc',aggfunc= np.sum)
ubitable

,mcp
sitc,
0011,40.0
0012,30.0
0013,17.0
0014,36.0
0015,21.0
...,...
9310,65.0
9410,61.0
9510,21.0


In [28]:
ubitable.sort_values(by='mcp').head(10) #top 10 products

,mcp
sitc,
7521,3.0
4245,4.0
2784,5.0
8933,5.0
7628,6.0
7187,6.0
2860,6.0
5843,7.0
8821,7.0


# PRODY 

Merge GDP per Capita

In [29]:
gdppercap

,country,gdp
0,ABW,0.000000
1,AFG,524.162881
2,AGO,3289.646664
3,ALB,5284.380184
4,AND,41793.055258
...,...,...
204,WSM,4183.408032
205,YEM,968.159048
206,ZAF,6374.028196
207,ZMB,1556.334482


In [30]:
gdp = pd.merge(data,gdppercap,on='country', how='left').fillna(0)
gdp

,country,sitc,export,cntry_export,prod_export,world_export,rca,mcp,diversity,ubiquity,gdp
0,ABW,0011,0.0,2.128365e+08,9.295920e+09,1.875183e+13,0.000000,0.0,42.0,40.0,0.000000
1,ABW,0012,0.0,2.128365e+08,1.788313e+09,1.875183e+13,0.000000,0.0,42.0,30.0,0.000000
2,ABW,0013,0.0,2.128365e+08,3.763658e+09,1.875183e+13,0.000000,0.0,42.0,17.0,0.000000
3,ABW,0014,74109.0,2.128365e+08,3.197278e+09,1.875183e+13,2.042153,1.0,42.0,36.0,0.000000
4,ABW,0015,23826.0,2.128365e+08,2.954505e+09,1.875183e+13,0.710500,0.0,42.0,21.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,2.193997e+09,7.072683e+11,1.875183e+13,1.248627,1.0,79.0,65.0,1683.740577
181008,ZWE,9410,746033.0,2.193997e+09,1.182526e+09,1.875183e+13,5.392063,1.0,79.0,61.0,1683.740577
181009,ZWE,9510,2241.0,2.193997e+09,1.344083e+10,1.875183e+13,0.001425,0.0,79.0,21.0,1683.740577
181010,ZWE,9610,0.0,2.193997e+09,4.515294e+08,1.875183e+13,0.000000,0.0,79.0,35.0,1683.740577


In [31]:
gdp.loc[gdp.country=='AUS']

,country,sitc,export,cntry_export,prod_export,world_export,rca,mcp,diversity,ubiquity,gdp
11505,AUS,0011,9.682510e+08,2.467784e+11,9.295920e+09,1.875183e+13,7.914657,1.0,95.0,40.0,57395.919466
11506,AUS,0012,1.044856e+08,2.467784e+11,1.788313e+09,1.875183e+13,4.439659,1.0,95.0,30.0,57395.919466
11507,AUS,0013,0.000000e+00,2.467784e+11,3.763658e+09,1.875183e+13,0.000000,0.0,95.0,17.0,57395.919466
11508,AUS,0014,7.960080e+06,2.467784e+11,3.197278e+09,1.875183e+13,0.189179,0.0,95.0,36.0,57395.919466
11509,AUS,0015,1.318013e+08,2.467784e+11,2.954505e+09,1.875183e+13,3.389780,1.0,95.0,21.0,57395.919466
...,...,...,...,...,...,...,...,...,...,...,...
12267,AUS,9310,2.823238e+10,2.467784e+11,7.072683e+11,1.875183e+13,3.033191,1.0,95.0,65.0,57395.919466
12268,AUS,9410,4.958638e+06,2.467784e+11,1.182526e+09,1.875183e+13,0.318631,0.0,95.0,61.0,57395.919466
12269,AUS,9510,7.653716e+07,2.467784e+11,1.344083e+10,1.875183e+13,0.432696,0.0,95.0,21.0,57395.919466
12270,AUS,9610,6.109780e+07,2.467784e+11,4.515294e+08,1.875183e+13,10.281964,1.0,95.0,35.0,57395.919466


In [32]:
new = gdp[["sitc","export","rca","mcp","diversity","ubiquity","gdp","country"]]
new

,sitc,export,rca,mcp,diversity,ubiquity,gdp,country
0,0011,0.0,0.000000,0.0,42.0,40.0,0.000000,ABW
1,0012,0.0,0.000000,0.0,42.0,30.0,0.000000,ABW
2,0013,0.0,0.000000,0.0,42.0,17.0,0.000000,ABW
3,0014,74109.0,2.042153,1.0,42.0,36.0,0.000000,ABW
4,0015,23826.0,0.710500,0.0,42.0,21.0,0.000000,ABW
...,...,...,...,...,...,...,...,...
181007,9310,103325888.0,1.248627,1.0,79.0,65.0,1683.740577,ZWE
181008,9410,746033.0,5.392063,1.0,79.0,61.0,1683.740577,ZWE
181009,9510,2241.0,0.001425,0.0,79.0,21.0,1683.740577,ZWE
181010,9610,0.0,0.000000,0.0,79.0,35.0,1683.740577,ZWE


PRODY

In [33]:
#PRODY DENOMINATOR 
new["prod_denom"] = new[["rca"]].groupby(new["sitc"]).transform(np.sum)
#PRODY numerator
new["prod_num"] = (new['rca']*new['gdp']).groupby(new['sitc']).transform(np.sum)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
new["PRODY"] = np.divide(new["prod_num"],new["prod_denom"])
new

,sitc,export,rca,mcp,diversity,ubiquity,gdp,country,prod_denom,prod_num,PRODY
0,0011,0.0,0.000000,0.0,42.0,40.0,0.000000,ABW,515.084188,4.508856e+06,8753.628944
1,0012,0.0,0.000000,0.0,42.0,30.0,0.000000,ABW,7848.604024,9.921081e+06,1264.056744
2,0013,0.0,0.000000,0.0,42.0,17.0,0.000000,ABW,102.291751,4.430893e+06,43316.228870
3,0014,74109.0,2.042153,1.0,42.0,36.0,0.000000,ABW,191.558438,2.428641e+06,12678.329831
4,0015,23826.0,0.710500,0.0,42.0,21.0,0.000000,ABW,130.828015,4.949987e+06,37835.836856
...,...,...,...,...,...,...,...,...,...,...,...
181007,9310,103325888.0,1.248627,1.0,79.0,65.0,1683.740577,ZWE,257.422570,3.698736e+06,14368.343827
181008,9410,746033.0,5.392063,1.0,79.0,61.0,1683.740577,ZWE,3757.931518,1.691542e+07,4501.257404
181009,9510,2241.0,0.001425,0.0,79.0,21.0,1683.740577,ZWE,117.842563,2.601498e+06,22076.051544
181010,9610,0.0,0.000000,0.0,79.0,35.0,1683.740577,ZWE,1606.965642,9.798515e+06,6097.525917


In [35]:
#display PRODY FOR EACH COUNTRY
new.loc[:,['country','PRODY']]

,country,PRODY
0,ABW,8753.628944
1,ABW,1264.056744
2,ABW,43316.228870
3,ABW,12678.329831
4,ABW,37835.836856
...,...,...
181007,ZWE,14368.343827
181008,ZWE,4501.257404
181009,ZWE,22076.051544
181010,ZWE,6097.525917


In [36]:
prody_table = pd.pivot_table(new,values='PRODY',index='country',columns='sitc').fillna(0)
prody_table

sitc,0011,0012,0013,0014,0015,0111,0112,0113,0114,0115,0116,0118,0121,0129,0141,0142,0149,0223,0224,0230,0240,0251,0252,0341,0342,0343,0344,0350,0360,0371,0372,0411,0412,0421,0422,0430,0440,0451,0452,0459,...,8841,8842,8851,8852,8921,8922,8924,8928,8931,8932,8933,8935,8939,8941,8942,8946,8947,8951,8952,8959,8960,8972,8973,8974,8981,8982,8983,8989,8991,8993,8994,8996,8997,8998,8999,9310,9410,9510,9610,9710
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ABW,8753.628944,1264.056744,43316.22887,12678.329831,37835.836856,16498.43384,22643.039446,37366.739406,14903.68498,9331.647996,24014.470057,32020.221298,21079.656282,15370.331617,31621.893576,17740.629253,16137.909156,20989.860485,30319.58433,34477.640303,27267.326571,15486.408529,26075.17927,10982.684439,6786.556429,25077.367708,14691.326549,19390.647545,9890.493439,9028.595668,7923.115817,23835.332227,14208.362166,5567.812526,5678.12512,16455.326691,9316.708828,18904.781062,31015.126227,5011.867653,...,22480.037677,28255.3942,48609.042768,25339.570206,9106.553559,20290.317947,13987.751209,22023.966215,17642.101606,17497.30733,16797.822249,32446.933061,15747.801555,24808.396518,17348.139931,19966.726742,13780.515884,8730.88628,20086.624915,24785.93798,5056.216265,24373.541861,31760.870056,34750.758917,3451.815992,17482.707529,23128.754685,12109.031247,14155.514789,16195.78341,10539.79582,25200.192038,13442.611045,21975.819327,2523.980566,14368.343827,4501.257404,22076.051544,6097.525917,6229.630482
AFG,8753.628944,1264.056744,43316.22887,12678.329831,37835.836856,16498.43384,22643.039446,37366.739406,14903.68498,9331.647996,24014.470057,32020.221298,21079.656282,15370.331617,31621.893576,17740.629253,16137.909156,20989.860485,30319.58433,34477.640303,27267.326571,15486.408529,26075.17927,10982.684439,6786.556429,25077.367708,14691.326549,19390.647545,9890.493439,9028.595668,7923.115817,23835.332227,14208.362166,5567.812526,5678.12512,16455.326691,9316.708828,18904.781062,31015.126227,5011.867653,...,22480.037677,28255.3942,48609.042768,25339.570206,9106.553559,20290.317947,13987.751209,22023.966215,17642.101606,17497.30733,16797.822249,32446.933061,15747.801555,24808.396518,17348.139931,19966.726742,13780.515884,8730.88628,20086.624915,24785.93798,5056.216265,24373.541861,31760.870056,34750.758917,3451.815992,17482.707529,23128.754685,12109.031247,14155.514789,16195.78341,10539.79582,25200.192038,13442.611045,21975.819327,2523.980566,14368.343827,4501.257404,22076.051544,6097.525917,6229.630482
AGO,8753.628944,1264.056744,43316.22887,12678.329831,37835.836856,16498.43384,22643.039446,37366.739406,14903.68498,9331.647996,24014.470057,32020.221298,21079.656282,15370.331617,31621.893576,17740.629253,16137.909156,20989.860485,30319.58433,34477.640303,27267.326571,15486.408529,26075.17927,10982.684439,6786.556429,25077.367708,14691.326549,19390.647545,9890.493439,9028.595668,7923.115817,23835.332227,14208.362166,5567.812526,5678.12512,16455.326691,9316.708828,18904.781062,31015.126227,5011.867653,...,22480.037677,28255.3942,48609.042768,25339.570206,9106.553559,20290.317947,13987.751209,22023.966215,17642.101606,17497.30733,16797.822249,32446.933061,15747.801555,24808.396518,17348.139931,19966.726742,13780.515884,8730.88628,20086.624915,24785.93798,5056.216265,24373.541861,31760.870056,34750.758917,3451.815992,17482.707529,23128.754685,12109.031247,14155.514789,16195.78341,10539.79582,25200.192038,13442.611045,21975.819327,2523.980566,14368.343827,4501.257404,22076.051544,6097.525917,6229.630482
AIA,8753.628944,1264.056744,43316.22887,12678.329831,37835.836856,16498.43384,22643.039446,37366.739406,14903.68498,9331.647996,24014.470057,32020.221298,21079.656282,15370.331617,31621.893576,17740.629253,16137.909156,20989.860485,30319.58433,34477.640303,27267.326571,15486.408529,26075.17927,10982.684439,6786.556429,25077.367708,14691.326549,19390.647545,9890.493439,9028.595668,7923.115817,23835.332227,14208.362166,5567.812526,5678.12512,16455.3266

#Density

*   Proximity
*   Density







**Proximity for product**

In [37]:
def proximity_matrix_numpy(mcp_table, fillna=False):
    products = sorted(list(mcp_table.columns))
    num_products = len(products)
    proximity = np.empty((num_products, num_products))
    col_sums = mcp_table.sum().values  
    data = mcp_table.T.to_numpy()                  #This generates a c x p numpy array
    for index1 in range(0,num_products):
        for index2 in range(0,num_products):
            if index2 > index1:
                continue
            numerator = (data[index1] * data[index2]).sum()
            denominator = max(col_sums[index1], col_sums[index2])
            if denominator == 0.0:
                cond_prob = np.nan
            else:
                cond_prob = numerator / denominator
            proximity[index1][index2] = cond_prob
            proximity[index2][index1] = cond_prob
    # Return DataFrame Representation #
    proximity = pd.DataFrame(proximity, index=products, columns=products)
    proximity.index.name = 'productcode1'
    proximity.columns.name = 'productcode2'
    if fillna:
        proximity = proximity.fillna(0.0)
    return proximity

In [38]:
prox = proximity_matrix_numpy(mcp_table)
prox

productcode2,0011,0012,0013,0014,0015,0111,0112,0113,0114,0115,0116,0118,0121,0129,0141,0142,0149,0223,0224,0230,0240,0251,0252,0341,0342,0343,0344,0350,0360,0371,0372,0411,0412,0421,0422,0430,0440,0451,0452,0459,...,8841,8842,8851,8852,8921,8922,8924,8928,8931,8932,8933,8935,8939,8941,8942,8946,8947,8951,8952,8959,8960,8972,8973,8974,8981,8982,8983,8989,8991,8993,8994,8996,8997,8998,8999,9310,9410,9510,9610,9710
productcode1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,1.000000,0.400000,0.350000,0.425000,0.225000,0.450000,0.350000,0.275000,0.375000,0.350000,0.300000,0.350000,0.200000,0.250000,0.275000,0.500000,0.525000,0.533333,0.300000,0.225000,0.425000,0.450000,0.300000,0.158730,0.151899,0.275000,0.188679,0.260870,0.183333,0.244444,0.185185,0.350000,0.350000,0.200000,0.100000,0.375000,0.300000,0.300000,0.350000,0.325000,...,0.100000,0.175000,0.025000,0.075000,0.250000,0.325000,0.100000,0.250000,0.396552,0.400000,0.000000,0.225000,0.375000,0.150000,0.150000,0.325000,0.275000,0.150000,0.125000,0.175000,0.050000,0.150000,0.050000,0.175000,0.100000,0.050000,0.125000,0.200000,0.200000,0.375000,0.125000,0.175000,0.125000,0.075000,0.075000,0.200000,0.311475,0.325000,0.225000,0.149254
0012,0.400000,1.000000,0.133333,0.194444,0.100000,0.133333,0.400000,0.100000,0.200000,0.200000,0.133333,0.166667,0.100000,0.033333,0.066667,0.200000,0.179487,0.177778,0.090909,0.066667,0.162162,0.230769,0.066667,0.158730,0.126582,0.100000,0.094340,0.195652,0.150000,0.155556,0.111111,0.266667,0.200000,0.166667,0.066667,0.266667,0.266667,0.066667,0.200000,0.290323,...,0.066667,0.000000,0.100000,0.033333,0.161290,0.166667,0.000000,0.068182,0.172414,0.166667,0.000000,0.100000,0.162162,0.033333,0.033333,0.212121,0.064516,0.100000,0.033333,0.133333,0.062500,0.066667,0.100000,0.066667,0.066667,0.033333,0.100000,0.066667,0.133333,0.150000,0.066667,0.066667,0.033333,0.033333,0.066667,0.138462,0.213115,0.200000,0.057143,0.179104
0013,0.350000,0.133333,1.000000,0.277778,0.285714,0.214286,0.133333,0.470588,0.285714,0.240000,0.178571,0.285714,0.470588,0.238095,0.300000,0.300000,0.256410,0.333333,0.212121,0.300000,0.270270,0.282051,0.450000,0.095238,0.088608,0.200000,0.132075,0.195652,0.100000,0.177778,0.129630,0.291667,0.250000,0.115385,0.050000,0.304348,0.083333,0.470588,0.450000,0.096774,...,0.117647,0.157895,0.000000,0.058824,0.193548,0.310345,0.117647,0.159091,0.241379,0.344828,0.000000,0.235294,0.216216,0.176471,0.125000,0.242424,0.161290,0.222222,0.200000,0.285714,0.000000,0.090909,0.071429,0.055556,0.117647,0.117647,0.181818,0.200000,0.217391,0.250000,0.117647,0.137931,0.185185,0.176471,0.095238,0.061538,0.163934,0.285714,0.085714,0.029851
0014,0.425000,0.194444,0.277778,1.000000,0.250000,0.250000,0.222222,0.305556,0.388889,0.305556,0.305556,0.361111,0.305556,0.194444,0.250000,0.575000,0.435897,0.422222,0.222222,0.250000,0.405405,0.564103,0.277778,0.174603,0.151899,0.300000,0.188679,0.239130,0.116667,0.222222,0.166667,0.277778,0.194444,0.166667,0.055556,0.277778,0.222222,0.250000,0.194444,0.166667,...,0.083333,0.138889,0.083333,0.083333,0.305556,0.416667,0.166667,0.340909,0.396552,0.388889,0.000000,0.222222,0.405405,0.138889,0.138889,0.305556,0.222222,0.166667,0.194444,0.277778,0.083333,0.166667,0.194444,0.222222,0.083333,0.111111,0.222222,0.194444,0.194444,0.450000,0.055556,0.305556,0.222222,0.166667,0.055556,0.123077,0.295082,0.194444,0.166667,0.074627
0015,0.225000,0.100000,0.285714,0.250000,1.000000,0.285714,0.300000,0.333333,0.285714,0.320000,0.392857,0.285714,0.285714,0.190476,0.238095,0.200000,0.230769,0.244444,0.363636,0.428571,0.324324,0.179487,0.285714,0.111111,0.113924,0.150000,0.113208,0.152174,0.100000,0.088889,0.092593,0.250000,0.285714,0.115385,0.095238,0.260870,0.166667,0.095238,0.190476,0.129032,...,0.190476,0.238095,0.190476,0.095238,0.161290,0.275862,0.142857,0.250000,0.155172,0.137931,0.047619,0.190476,0.135135,0.095238,0.083333,0.212121,0.096774,0.142857,0.142857,0.142857,0.218750,0.136364,0.1785

**density**

In [39]:
mcp_t = mcp_table.T
mcp_t

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
sitc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0012,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
0013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0014,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
0015,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9310,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9410,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
9510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [40]:
den = prox @ mcp_t
den

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
productcode1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,8.945180,13.438852,2.660732,11.041368,26.563821,20.657187,6.936970,5.219138,10.955099,31.637657,19.011720,6.304426,3.425000,4.803296,7.549149,24.731234,76.091022,5.560832,7.784720,65.333100,11.887162,1.981855,5.583614,12.427492,67.125526,10.939483,7.325080,50.158638,12.840035,50.060889,11.760529,4.691106,14.412545,35.196157,21.375935,1.807426,6.106343,2.175163,6.350886,4.510588,...,56.841924,64.105290,54.395548,21.220055,5.370056,5.466593,26.034057,11.219937,2.449169,14.277209,52.279223,7.542960,16.298690,5.774353,4.508065,9.208244,7.745693,32.581350,66.729925,0.726899,27.397526,36.836527,29.794192,51.677632,7.089846,27.899748,58.909879,14.777407,10.372231,6.059269,5.471156,11.873180,33.359746,4.871313,2.005814,23.931908,11.719810,36.198809,18.190391,18.062043
0012,4.940801,12.839288,2.280681,5.422231,17.666531,12.424643,4.134919,2.896106,8.151812,20.464643,15.223902,3.967944,2.849504,2.616912,4.439167,16.636257,35.610898,4.523964,4.868396,33.603403,7.048003,1.401699,5.438724,9.108368,36.292497,7.085894,4.677815,25.436755,7.052250,26.315992,7.232588,2.964071,9.792736,18.650045,12.056814,1.264684,3.371433,1.402187,4.053948,3.426958,...,27.298265,30.714207,25.894833,12.464162,3.063762,3.304106,19.365635,6.068527,2.650827,8.859536,27.432292,6.484271,10.057431,5.131527,2.697749,5.607831,4.115564,21.290204,41.355666,0.250392,14.266106,26.616901,18.602328,26.541371,3.803250,14.767774,31.192674,11.733345,5.588246,3.858521,3.692347,6.452488,20.454845,3.388530,1.118258,13.595848,10.749031,22.646905,11.282893,12.648125
0013,6.950318,7.323050,1.589393,8.209763,14.421034,16.201442,6.331443,4.077173,7.163696,20.845297,11.385164,4.672154,2.576806,2.985157,4.963606,14.936162,60.628010,3.450887,4.204916,62.722861,6.771678,1.541994,2.613570,6.895296,44.918431,8.263454,4.932132,32.089320,9.688487,39.635809,7.555063,3.800177,7.792599,24.340898,15.166989,1.209300,3.757570,1.933145,3.340857,2.314178,...,46.094648,47.704342,46.514672,13.749992,3.972963,3.493565,16.133380,8.533042,0.873598,8.450512,46.354827,3.682457,11.774486,4.395073,3.075598,5.998635,4.871122,22.106221,50.519388,0.494322,30.194059,24.209295,19.435857,35.038393,5.807268,18.052278,49.552453,9.848518,7.782502,3.960286,3.062863,9.794739,24.469203,2.796689,1.420756,16.464008,6.117556,24.388987,9.995867,10.770756
0014,10.913901,12.008950,2.460100,11.364448,26.029298,22.721403,7.469389,6.021797,10.525736,30.505483,19.608308,6.764339,3.434480,5.061536,7.604875,19.574871,78.512968,5.679123,7.200354,74.147888,11.228531,2.338979,5.141117,13.802487,67.654678,12.243504,6.706907,46.131739,13.880766,51.568713,11.270353,5.147327,12.844562,34.205193,24.917810,1.934868,6.215971,2.314369,5.708629,3.818320,...,55.877341,62.175230,57.172901,24.227719,5.842441,5.506194,28.134928,11.551096,2.233066,14.246507,51.593476,6.866092,18.433031,6.531041,4.288392,9.810051,8.185621,35.057088,74.748564,0.707454,31.214403,36.947734,30.143929,46.470281,7.552885,24.822117,61.899023,14.937604,10.232313,6.212992,4.686157,11.829931,34.411652,4.334639,2.101113,23.380291,10.479799,35.733796,19.362085,17.672896
0015,7.190361,8.365538,1.559272,6.045598,11.811664,17.296120,5.654963,3.520020,6.104835,24.443388,10.338728,4.627273,2.935335,2.641875,4.041192,18.078866,45.276423,2.964997,3.920769,50.376729,5.664957,1.725073,2.751599,6.089010,32.713525,6.892372,4.470465,21.281716,6.669986,28.416137,7.261878,5.840887,8.533051,19.702314,15.670442,1.625850,2.822791,1.414658,3.474195,2.398759,...,26.621946,32.317915,32.959798,11.430918,3.377763,3.153717,14.472979,5.647469,1.680461,7.627850,29.241159,4.241765,8.

In [41]:
den_denom = pd.DataFrame(prox.sum())
den_denom

,0
productcode2,
0011,161.030653
0012,93.024237
0013,137.020442
0014,168.372615
0015,109.297822
...,...
9310,95.518728
9410,112.004779
9510,126.356486


In [42]:
density = np.divide(den,den_denom)
density

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
productcode1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,0.055550,0.083455,0.016523,0.068567,0.164961,0.128281,0.043079,0.032411,0.068031,0.196470,0.118063,0.039150,0.021269,0.029828,0.046880,0.153581,0.472525,0.034533,0.048343,0.405718,0.073819,0.012307,0.034674,0.077175,0.416849,0.067934,0.045489,0.311485,0.079737,0.310878,0.073033,0.029132,0.089502,0.218568,0.132745,0.011224,0.037920,0.013508,0.039439,0.028011,...,0.352988,0.398094,0.337796,0.131776,0.033348,0.033948,0.161671,0.069676,0.015209,0.088661,0.324654,0.046842,0.101215,0.035859,0.027995,0.057183,0.048101,0.202330,0.414393,0.004514,0.170139,0.228755,0.185022,0.320918,0.044028,0.173257,0.365830,0.091768,0.064412,0.037628,0.033976,0.073732,0.207164,0.030251,0.012456,0.148617,0.072780,0.224795,0.112962,0.112165
0012,0.053113,0.138021,0.024517,0.058288,0.189913,0.133563,0.044450,0.031133,0.087631,0.219993,0.163655,0.042655,0.030632,0.028132,0.047721,0.178838,0.382813,0.048632,0.052335,0.361233,0.075765,0.015068,0.058466,0.097914,0.390140,0.076173,0.050286,0.273442,0.075811,0.282894,0.077750,0.031863,0.105271,0.200486,0.129609,0.013595,0.036243,0.015073,0.043579,0.036839,...,0.293453,0.330174,0.278367,0.133988,0.032935,0.035519,0.208178,0.065236,0.028496,0.095239,0.294894,0.069705,0.108116,0.055163,0.029000,0.060284,0.044242,0.228867,0.444569,0.002692,0.153359,0.286129,0.199973,0.285317,0.040885,0.158752,0.335318,0.126132,0.060073,0.041479,0.039692,0.069364,0.219887,0.036426,0.012021,0.146154,0.115551,0.243452,0.121290,0.135966
0013,0.050725,0.053445,0.011600,0.059916,0.105247,0.118241,0.046208,0.029756,0.052282,0.152133,0.083091,0.034098,0.018806,0.021786,0.036225,0.109007,0.442474,0.025185,0.030688,0.457763,0.049421,0.011254,0.019074,0.050323,0.327823,0.060308,0.035996,0.234194,0.070708,0.289269,0.055138,0.027734,0.056872,0.177644,0.110691,0.008826,0.027423,0.014108,0.024382,0.016889,...,0.336407,0.348155,0.339472,0.100350,0.028995,0.025497,0.117744,0.062276,0.006376,0.061673,0.338306,0.026875,0.085932,0.032076,0.022446,0.043779,0.035550,0.161335,0.368700,0.003608,0.220362,0.176684,0.141846,0.255717,0.042382,0.131749,0.361643,0.071876,0.056798,0.028903,0.022353,0.071484,0.178581,0.020411,0.010369,0.120157,0.044647,0.177995,0.072952,0.078607
0014,0.064820,0.071324,0.014611,0.067496,0.154593,0.134947,0.044362,0.035765,0.062515,0.181178,0.116458,0.040175,0.020398,0.030062,0.045167,0.116259,0.466305,0.033729,0.042764,0.440380,0.066689,0.013892,0.030534,0.081976,0.401815,0.072717,0.039834,0.273986,0.082441,0.306277,0.066937,0.030571,0.076287,0.203152,0.147992,0.011492,0.036918,0.013746,0.033905,0.022678,...,0.331867,0.369272,0.339562,0.143893,0.034699,0.032702,0.167099,0.068604,0.013263,0.084613,0.306424,0.040779,0.109478,0.038789,0.025470,0.058264,0.048616,0.208211,0.443947,0.004202,0.185389,0.219440,0.179031,0.275997,0.044858,0.147424,0.367631,0.088718,0.060772,0.036900,0.027832,0.070260,0.204378,0.025744,0.012479,0.138860,0.062242,0.212230,0.114995,0.104963
0015,0.065787,0.076539,0.014266,0.055313,0.108069,0.158248,0.051739,0.032206,0.055855,0.223640,0.094592,0.042336,0.026856,0.024171,0.036974,0.165409,0.414248,0.027128,0.035872,0.460912,0.051830,0.015783,0.025175,0.055710,0.299306,0.063060,0.040902,0.194713,0.061026,0.259988,0.066441,0.053440,0.078072,0.180263,0.143374,0.014875,0.025827,0.012943,0.031786,0.021947,...,0.243573,0.295687,0.301560,0.104585,0.030904,0.028854,0.132418,0.051670,0.015375,0.069790,0.267537,0.038809,0.074542,0.035659,0.022676,0.048497,0.035198,0.147214,0.312358,0.005277,0.204413,0.176896,0.149983,0.225155,0.048952,0.153226,0.450127,0.074988,0.052775,0.029577,0.025293,0.074161,0

In [43]:
density.loc[:,['MYS']]

country,MYS
productcode1,
0011,0.171006
0012,0.157243
0013,0.188531
0014,0.183386
0015,0.171309
...,...
9310,0.181143
9410,0.163976
9510,0.163640


# Distance

In [44]:
rev_mcp = 1 - mcp_t
rev_mcp

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
sitc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0012,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
0013,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0014,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
0015,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9310,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
9410,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
9510,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0


In [45]:
dis = prox @ rev_mcp
dis

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
productcode1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,152.085473,147.591801,158.369921,149.989285,134.466832,140.373466,154.093683,155.811515,150.075554,129.392996,142.018933,154.726227,157.605653,156.227357,153.481504,136.299419,84.939632,155.469821,153.245933,95.697553,149.143491,159.048798,155.447039,148.603161,93.905127,150.091170,153.705573,110.872016,148.190618,110.969764,149.270124,156.339547,146.618108,125.834496,139.654718,159.223227,154.924310,158.855490,154.679767,156.520065,...,104.188729,96.925363,106.635105,139.810598,155.660597,155.564060,134.996596,149.810716,158.581484,146.753444,108.751430,153.487693,144.731963,155.256300,156.522588,151.822409,153.284960,128.449303,94.300728,160.303754,133.633127,124.194126,131.236461,109.353021,153.940807,133.130905,102.120774,146.253246,150.658422,154.971384,155.559497,149.157473,127.670907,156.159340,159.024839,137.098745,149.310843,124.831844,142.840262,142.968610
0012,88.083436,80.184949,90.743556,87.602006,75.357706,80.599594,88.889318,90.128131,84.872425,72.559594,77.800335,89.056293,90.174733,90.407325,88.585070,76.387980,57.413339,88.500273,88.155841,59.420834,85.976234,91.622538,87.585513,83.915869,56.731740,85.938343,88.346422,67.587482,85.971987,66.708245,85.791649,90.060167,83.231501,74.374192,80.967423,91.759554,89.652804,91.622050,88.970289,89.597279,...,65.725972,62.310030,67.129404,80.560075,89.960475,89.720131,73.658603,86.955710,90.373410,84.164701,65.591945,86.539967,82.966806,87.892710,90.326488,87.416406,88.908673,71.734033,51.668571,92.773845,78.758131,66.407337,74.421909,66.482866,89.220987,78.256463,61.831563,81.290892,87.435991,89.165716,89.331890,86.571749,72.569392,89.635707,91.905979,79.428389,82.275206,70.377332,81.741344,80.376112
0013,130.070125,129.697392,135.431050,128.810679,122.599408,120.819001,130.688999,132.943269,129.856747,116.175145,125.635278,132.348288,134.443636,134.035285,132.056837,122.084281,76.392432,133.569555,132.815527,74.297581,130.248765,135.478449,134.406872,130.125146,92.102012,128.756989,132.088310,104.931122,127.331955,97.384633,129.465379,133.220265,129.227843,112.679544,121.853453,135.811142,133.262872,135.087298,133.679585,134.706265,...,90.925794,89.316101,90.505770,123.270450,133.047479,133.526877,120.887063,128.487400,136.146844,128.569930,90.665615,133.337986,125.245956,132.625370,133.944844,131.021808,132.149320,114.914221,86.501054,136.526120,106.826384,112.811148,117.584585,101.982050,131.213175,118.968164,87.467989,127.171925,129.237940,133.060156,133.957579,127.225703,112.551239,134.223754,135.599686,120.556434,130.902886,112.631455,127.024576,126.249687
0014,157.458714,156.363665,165.912515,157.008167,142.343317,145.651212,160.903226,162.350818,157.846879,137.867132,148.764307,161.608276,164.938135,163.311079,160.767740,148.797744,89.859647,162.693492,161.172261,94.224727,157.144084,166.033636,163.231498,154.570128,100.717937,156.129111,161.665708,122.240876,154.491849,116.803902,157.102262,163.225288,155.528053,134.167422,143.454805,166.437747,162.156644,166.058246,162.663986,164.554295,...,112.495274,106.197385,111.199714,144.144896,162.530174,162.866421,140.237687,156.821519,166.139549,154.126108,116.779139,161.506523,149.939584,161.841574,164.084223,158.562564,160.186994,133.315527,93.624051,167.665161,137.158212,131.424881,138.228686,121.902334,160.819730,143.550498,106.473592,153.435011,158.140302,162.159623,163.686458,156.542684,133.960963,164.037976,166.271502,144.992324,157.892816,132.638819,149.010530,150.699719
0015,102.107461,100.932285,107.738550,103.252224,97.486158,92.001702,103.642859,105.777802,103.192987,84.854434,98.9

In [46]:
distance = np.divide(dis,den_denom)
distance

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
productcode1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,0.944450,0.916545,0.983477,0.931433,0.835039,0.871719,0.956921,0.967589,0.931969,0.803530,0.881937,0.960850,0.978731,0.970172,0.953120,0.846419,0.527475,0.965467,0.951657,0.594282,0.926181,0.987693,0.965326,0.922825,0.583151,0.932066,0.954511,0.688515,0.920263,0.689122,0.926967,0.970868,0.910498,0.781432,0.867255,0.988776,0.962080,0.986492,0.960561,0.971989,...,0.647012,0.601906,0.662204,0.868224,0.966652,0.966052,0.838329,0.930324,0.984791,0.911339,0.675346,0.953158,0.898785,0.964141,0.972005,0.942817,0.951899,0.797670,0.585607,0.995486,0.829861,0.771245,0.814978,0.679082,0.955972,0.826743,0.634170,0.908232,0.935588,0.962372,0.966024,0.926268,0.792836,0.969749,0.987544,0.851383,0.927220,0.775205,0.887038,0.887835
0012,0.946887,0.861979,0.975483,0.941712,0.810087,0.866437,0.955550,0.968867,0.912369,0.780007,0.836345,0.957345,0.969368,0.971868,0.952279,0.821162,0.617187,0.951368,0.947665,0.638767,0.924235,0.984932,0.941534,0.902086,0.609860,0.923827,0.949714,0.726558,0.924189,0.717106,0.922250,0.968137,0.894729,0.799514,0.870391,0.986405,0.963757,0.984927,0.956421,0.963161,...,0.706547,0.669826,0.721633,0.866012,0.967065,0.964481,0.791822,0.934764,0.971504,0.904761,0.705106,0.930295,0.891884,0.944837,0.971000,0.939716,0.955758,0.771133,0.555431,0.997308,0.846641,0.713871,0.800027,0.714683,0.959115,0.841248,0.664682,0.873868,0.939927,0.958521,0.960308,0.930636,0.780113,0.963574,0.987979,0.853846,0.884449,0.756548,0.878710,0.864034
0013,0.949275,0.946555,0.988400,0.940084,0.894753,0.881759,0.953792,0.970244,0.947718,0.847867,0.916909,0.965902,0.981194,0.978214,0.963775,0.890993,0.557526,0.974815,0.969312,0.542237,0.950579,0.988746,0.980926,0.949677,0.672177,0.939692,0.964004,0.765806,0.929292,0.710731,0.944862,0.972266,0.943128,0.822356,0.889309,0.991174,0.972577,0.985892,0.975618,0.983111,...,0.663593,0.651845,0.660528,0.899650,0.971005,0.974503,0.882256,0.937724,0.993624,0.938327,0.661694,0.973125,0.914068,0.967924,0.977554,0.956221,0.964450,0.838665,0.631300,0.996392,0.779638,0.823316,0.858154,0.744283,0.957618,0.868251,0.638357,0.928124,0.943202,0.971097,0.977647,0.928516,0.821419,0.979589,0.989631,0.879843,0.955353,0.822005,0.927048,0.921393
0014,0.935180,0.928676,0.985389,0.932504,0.845407,0.865053,0.955638,0.964235,0.937485,0.818822,0.883542,0.959825,0.979602,0.969938,0.954833,0.883741,0.533695,0.966271,0.957236,0.559620,0.933311,0.986108,0.969466,0.918024,0.598185,0.927283,0.960166,0.726014,0.917559,0.693723,0.933063,0.969429,0.923713,0.796848,0.852008,0.988508,0.963082,0.986254,0.966095,0.977322,...,0.668133,0.630728,0.660438,0.856107,0.965301,0.967298,0.832901,0.931396,0.986737,0.915387,0.693576,0.959221,0.890522,0.961211,0.974530,0.941736,0.951384,0.791789,0.556053,0.995798,0.814611,0.780560,0.820969,0.724003,0.955142,0.852576,0.632369,0.911282,0.939228,0.963100,0.972168,0.929740,0.795622,0.974256,0.987521,0.861140,0.937758,0.787770,0.885005,0.895037
0015,0.934213,0.923461,0.985734,0.944687,0.891931,0.841752,0.948261,0.967794,0.944145,0.776360,0.905408,0.957664,0.973144,0.975829,0.963026,0.834591,0.585752,0.972872,0.964128,0.539088,0.948170,0.984217,0.974825,0.944290,0.700694,0.936940,0.959098,0.805287,0.938974,0.740012,0.933559,0.946560,0.921928,0.819737,0.856626,0.985125,0.974173,0.987057,0.968214,0.978053,...,0.756427,0.704313,0.698440,0.895415,0.969096,0.971146,0.867582,0.948330,0.984625,0.930210,0.732463,0.961191,0.925458,0.964341,0.977324,0.951503,0.964802,0.852786,0.687642,0.994723,0.795587,0.823104,0.850017,0.774845,0.951048,0.846774,0.549873,0.925012,0.947225,0.970423,0.974707,0.925839,0

# **Open forest**


In [47]:
prody_t = prody_table.T
prody_t

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
sitc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,...,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944,8753.628944
0012,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,...,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744,1264.056744
0013,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,...,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870,43316.228870
0014,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,...,12678.329831,12678.329831,12678.329831,12678.329831,12678.329831,1

In [48]:
forest = pd.DataFrame((density*rev_mcp*prody_t).sum())
forest

,0
country,
ABW,6.549819e+05
AFG,7.908953e+05
AGO,1.718342e+05
AIA,7.770966e+05
ALB,1.399363e+06
...,...
WSM,1.386716e+06
YEM,6.122523e+05
ZAF,2.053488e+06


#EXPY

In [49]:
#EXPY
new["expy_denom"] = new[["rca"]].groupby(new["country"]).transform(np.sum)
new["expy_num"]= new['rca']*new['PRODY']
new

,sitc,export,rca,mcp,diversity,ubiquity,gdp,country,prod_denom,prod_num,PRODY,expy_denom,expy_num
0,0011,0.0,0.000000,0.0,42.0,40.0,0.000000,ABW,515.084188,4.508856e+06,8753.628944,342.517097,0.000000
1,0012,0.0,0.000000,0.0,42.0,30.0,0.000000,ABW,7848.604024,9.921081e+06,1264.056744,342.517097,0.000000
2,0013,0.0,0.000000,0.0,42.0,17.0,0.000000,ABW,102.291751,4.430893e+06,43316.228870,342.517097,0.000000
3,0014,74109.0,2.042153,1.0,42.0,36.0,0.000000,ABW,191.558438,2.428641e+06,12678.329831,342.517097,25891.083982
4,0015,23826.0,0.710500,0.0,42.0,21.0,0.000000,ABW,130.828015,4.949987e+06,37835.836856,342.517097,26882.362792
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181007,9310,103325888.0,1.248627,1.0,79.0,65.0,1683.740577,ZWE,257.422570,3.698736e+06,14368.343827,2678.043704,17940.700118
181008,9410,746033.0,5.392063,1.0,79.0,61.0,1683.740577,ZWE,3757.931518,1.691542e+07,4501.257404,2678.043704,24271.064074
181009,9510,2241.0,0.001425,0.0,79.0,21.0,1683.740577,ZWE,117.842563,2.601498e+06,22076.051544,2678.043704,31.458990
181010,9610,0.0,0.000000,0.0,79.0,35.0,1683.740577,ZWE,1606.965642,9.798515e+06,6097.525917,2678.043704,0.000000


In [50]:
new["expy_"] = new['expy_num'].groupby(new['country']).transform(np.sum)
new

,sitc,export,rca,mcp,diversity,ubiquity,gdp,country,prod_denom,prod_num,PRODY,expy_denom,expy_num,expy_
0,0011,0.0,0.000000,0.0,42.0,40.0,0.000000,ABW,515.084188,4.508856e+06,8753.628944,342.517097,0.000000,5.863845e+06
1,0012,0.0,0.000000,0.0,42.0,30.0,0.000000,ABW,7848.604024,9.921081e+06,1264.056744,342.517097,0.000000,5.863845e+06
2,0013,0.0,0.000000,0.0,42.0,17.0,0.000000,ABW,102.291751,4.430893e+06,43316.228870,342.517097,0.000000,5.863845e+06
3,0014,74109.0,2.042153,1.0,42.0,36.0,0.000000,ABW,191.558438,2.428641e+06,12678.329831,342.517097,25891.083982,5.863845e+06
4,0015,23826.0,0.710500,0.0,42.0,21.0,0.000000,ABW,130.828015,4.949987e+06,37835.836856,342.517097,26882.362792,5.863845e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181007,9310,103325888.0,1.248627,1.0,79.0,65.0,1683.740577,ZWE,257.422570,3.698736e+06,14368.343827,2678.043704,17940.700118,9.244978e+06
181008,9410,746033.0,5.392063,1.0,79.0,61.0,1683.740577,ZWE,3757.931518,1.691542e+07,4501.257404,2678.043704,24271.064074,9.244978e+06
181009,9510,2241.0,0.001425,0.0,79.0,21.0,1683.740577,ZWE,117.842563,2.601498e+06,22076.051544,2678.043704,31.458990,9.244978e+06
181010,9610,0.0,0.000000,0.0,79.0,35.0,1683.740577,ZWE,1606.965642,9.798515e+06,6097.525917,2678.043704,0.000000,9.244978e+06


In [51]:
new["EXPY"] = new['expy_']/new['expy_denom']
new

,sitc,export,rca,mcp,diversity,ubiquity,gdp,country,prod_denom,prod_num,PRODY,expy_denom,expy_num,expy_,EXPY
0,0011,0.0,0.000000,0.0,42.0,40.0,0.000000,ABW,515.084188,4.508856e+06,8753.628944,342.517097,0.000000,5.863845e+06,17119.861705
1,0012,0.0,0.000000,0.0,42.0,30.0,0.000000,ABW,7848.604024,9.921081e+06,1264.056744,342.517097,0.000000,5.863845e+06,17119.861705
2,0013,0.0,0.000000,0.0,42.0,17.0,0.000000,ABW,102.291751,4.430893e+06,43316.228870,342.517097,0.000000,5.863845e+06,17119.861705
3,0014,74109.0,2.042153,1.0,42.0,36.0,0.000000,ABW,191.558438,2.428641e+06,12678.329831,342.517097,25891.083982,5.863845e+06,17119.861705
4,0015,23826.0,0.710500,0.0,42.0,21.0,0.000000,ABW,130.828015,4.949987e+06,37835.836856,342.517097,26882.362792,5.863845e+06,17119.861705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181007,9310,103325888.0,1.248627,1.0,79.0,65.0,1683.740577,ZWE,257.422570,3.698736e+06,14368.343827,2678.043704,17940.700118,9.244978e+06,3452.138534
181008,9410,746033.0,5.392063,1.0,79.0,61.0,1683.740577,ZWE,3757.931518,1.691542e+07,4501.257404,2678.043704,24271.064074,9.244978e+06,3452.138534
181009,9510,2241.0,0.001425,0.0,79.0,21.0,1683.740577,ZWE,117.842563,2.601498e+06,22076.051544,2678.043704,31.458990,9.244978e+06,3452.138534
181010,9610,0.0,0.000000,0.0,79.0,35.0,1683.740577,ZWE,1606.965642,9.798515e+06,6097.525917,2678.043704,0.000000,9.244978e+06,3452.138534


# ECI & PCI
-Calculation


In [52]:
sub = data.loc[:,["country","sitc","export","rca","mcp","diversity","ubiquity"]]
sub

,country,sitc,export,rca,mcp,diversity,ubiquity
0,ABW,0011,0.0,0.000000,0.0,42.0,40.0
1,ABW,0012,0.0,0.000000,0.0,42.0,30.0
2,ABW,0013,0.0,0.000000,0.0,42.0,17.0
3,ABW,0014,74109.0,2.042153,1.0,42.0,36.0
4,ABW,0015,23826.0,0.710500,0.0,42.0,21.0
...,...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,1.248627,1.0,79.0,65.0
181008,ZWE,9410,746033.0,5.392063,1.0,79.0,61.0
181009,ZWE,9510,2241.0,0.001425,0.0,79.0,21.0
181010,ZWE,9610,0.0,0.000000,0.0,79.0,35.0


In [53]:
sub["m_ubi"] = (sub["mcp"]/sub["ubiquity"]).fillna(0)
sub["m_div"] = (sub["mcp"]/sub["diversity"]).fillna(0)
sub

,country,sitc,export,rca,mcp,diversity,ubiquity,m_ubi,m_div
0,ABW,0011,0.0,0.000000,0.0,42.0,40.0,0.000000,0.000000
1,ABW,0012,0.0,0.000000,0.0,42.0,30.0,0.000000,0.000000
2,ABW,0013,0.0,0.000000,0.0,42.0,17.0,0.000000,0.000000
3,ABW,0014,74109.0,2.042153,1.0,42.0,36.0,0.027778,0.023810
4,ABW,0015,23826.0,0.710500,0.0,42.0,21.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
181007,ZWE,9310,103325888.0,1.248627,1.0,79.0,65.0,0.015385,0.012658
181008,ZWE,9410,746033.0,5.392063,1.0,79.0,61.0,0.016393,0.012658
181009,ZWE,9510,2241.0,0.001425,0.0,79.0,21.0,0.000000,0.000000
181010,ZWE,9610,0.0,0.000000,0.0,79.0,35.0,0.000000,0.000000


In [54]:
#display only MCP
mcp1 = pd.pivot_table(sub,values='m_div',index='country',columns='sitc').fillna(0)
mcp2 = pd.pivot_table(sub,values='m_ubi',index='country',columns='sitc').fillna(0)

In [55]:
#get matrix c*c and p*p
mcc= mcp1@mcp2.T
mpp = mcp2.T@ mcp1

In [56]:
eigvals,eigvecs = np.linalg.eig(mpp)
eigvecs = np.real(eigvecs)
# Get eigenvector corresponding to second largest eigenvalue
eig_index = eigvals.argsort()[-2]
kp = eigvecs[:, eig_index]
kc = mcp1 @ kp

In [57]:
divs_tab = (sub[["country", "mcp"]].groupby(by=["country"]).sum()).astype(np.float64)
divs= np.ravel(divs_tab)
divs
#print("div_tab",divs_tab)

array([ 42.,  65.,  16.,  44., 103.,  99.,  38.,  23.,  53., 127.,  79.,
        29.,  20.,  18.,  32.,  95., 288.,  27.,  31., 278.,  50.,  10.,
        25.,  58., 238.,  51.,  32., 169.,  54., 185.,  54.,  26.,  62.,
       143.,  86.,  11.,  24.,   9.,  24.,  21., 185.,  47., 130., 104.,
       332.,  64.,  40.,  18.,  16.,  21.,  91.,  10.,  31., 123.,  34.,
        49.,  15.,   7.,  86., 261., 325.,  40.,  47., 237.,  90.,  17.,
        57., 200.,  10.,   8., 331., 217.,  90., 170., 109.,   8., 305.,
        17.,  10.,  16., 207., 111.,  45.,  25.,  24.,  41.,   5.,   5.,
       172.,  39.,  16., 130.,  67.,  37., 173.,   0.,  93., 243.,  37.,
       204., 189., 273.,  33.,  79.,  71.,   7.,  57., 163., 332.,  55.,
       145., 218.,  80., 148., 112.,  82.,   9.,  25., 157.,  13.,  96.,
       172.,  22.,  10.,  36., 138.,  48., 255., 164., 209.,  85., 119.,
       123.,  78.,  16., 124.,  20.,  99.,  27.,  69., 114.,  90.,  40.,
         8.,  61.,  27.,  33., 107.,  42., 136.,  7

In [58]:
s1 = np.sign(np.corrcoef(divs, kc)[0, 1])
eci_t = s1 * kc
pci_t = s1 * kp

In [59]:
#ECI
eci = (eci_t- eci_t.mean())/eci_t.std()
eci = pd.DataFrame(eci)

In [60]:
#PCI
pci = (pci_t- pci_t.mean())/pci_t.std()
pci = pd.DataFrame(pci)
pci

,0
0,-0.297238
1,-1.463164
2,0.695724
3,0.051345
4,-0.225070
...,...
762,-0.270970
763,-1.307268
764,-0.059164
765,-0.121457


In [61]:
index1 = data[['sitc']].drop_duplicates()

In [62]:
pci = pd.concat([index1, pci],axis=1)
pci

,sitc,0
0,0011,-0.297238
1,0012,-1.463164
2,0013,0.695724
3,0014,0.051345
4,0015,-0.225070
...,...,...
762,9310,-0.270970
763,9410,-1.307268
764,9510,-0.059164
765,9610,-0.121457


In [63]:
pci.set_index('sitc')

,0
sitc,
0011,-0.297238
0012,-1.463164
0013,0.695724
0014,0.051345
0015,-0.225070
...,...
9310,-0.270970
9410,-1.307268
9510,-0.059164


In [ ]:
eci['rank'] = eci.rank(ascending=False)

In [ ]:
#ECI with ranking and country name
eci = pd.merge(eci,cty_list,on='country', how='left')
eci

,country,0,rank,Continent_Name,Country_Name
0,ABW,0.737734,63.0,North America,Aruba
1,AFG,-1.160829,203.0,Asia,"Afghanistan, Islamic Republic of"
2,AGO,-1.528365,221.0,Africa,"Angola, Republic of"
3,AIA,1.580401,13.0,North America,Anguilla
4,ALB,-0.283953,142.0,Europe,"Albania, Republic of"
...,...,...,...,...,...
239,WSM,0.144178,104.0,Oceania,"Samoa, Independent State of"
240,YEM,-1.435886,219.0,Asia,Yemen
241,ZAF,-0.114437,129.0,Africa,"South Africa, Republic of"
242,ZMB,-0.650563,172.0,Africa,"Zambia, Republic of"


In [ ]:
eci = eci.drop_duplicates(subset=['country'])

In [ ]:
#PCI Ranking
pci['rank'] = pci[0].rank(ascending=False)
pci.sort_values(by='rank')

,sitc,0,rank
566,7367,2.087947,1.0
724,8821,1.889096,2.0
725,8822,1.853704,3.0
313,5838,1.842475,4.0
257,5163,1.836827,5.0
...,...,...,...
196,2922,-2.450628,763.0
142,2631,-2.644727,764.0
114,2225,-2.645427,765.0
209,3330,-2.971170,766.0


# PCI ranking
*   Top 10 MOST complex product
*   Top 10 LEAST complex product





In [ ]:
pci['two']= pci['sitc'].astype(str).str[:2]
pci

,sitc,0,rank,two
0,0011,-0.297238,477.0,00
1,0012,-1.463164,696.0,00
2,0013,0.695724,221.0,00
3,0014,0.051345,395.0,00
4,0015,-0.225070,460.0,00
...,...,...,...,...
762,9310,-0.270970,469.0,93
763,9410,-1.307268,676.0,94
764,9510,-0.059164,426.0,95
765,9610,-0.121457,436.0,96


In [ ]:
pci_two = pd.merge(pci, SITC2, on= 'two', how='left')
pci_two

,sitc,0,rank,two,Commodity description
0,0011,-0.297238,477.0,00,Live animals chiefly for food
1,0012,-1.463164,696.0,00,Live animals chiefly for food
2,0013,0.695724,221.0,00,Live animals chiefly for food
3,0014,0.051345,395.0,00,Live animals chiefly for food
4,0015,-0.225070,460.0,00,Live animals chiefly for food
...,...,...,...,...,...
762,9310,-0.270970,469.0,93,"Special transactions, commodity not classified..."
763,9410,-1.307268,676.0,94,"Animals, live, nes, (including zoo animals, pe..."
764,9510,-0.059164,426.0,95,"Armoured fighting vehicles, war firearms, ammu..."
765,9610,-0.121457,436.0,96,"Coin (other than gold coin), not being legal t..."


In [ ]:
#pci_two.rename({0:"pci"},axis=1,inplace=True)

In [ ]:
#top 10 high complex products
pci_top = pci_two.sort_values(by='rank').head(10)
pci_top

,sitc,0,rank,two,Commodity description
566,7367,2.087947,1.0,73,Metalworking machinery
724,8821,1.889096,2.0,88,"Photographic equipment and supplies, optical g..."
725,8822,1.853704,3.0,88,"Photographic equipment and supplies, optical g..."
313,5838,1.842475,4.0,58,"Artificial resins and plastic materials, and c..."
257,5163,1.836827,5.0,51,Organic chemicals
564,7361,1.802222,6.0,73,Metalworking machinery
551,7251,1.767300,7.0,72,Machinery specialized for particular industries
503,6954,1.764681,8.0,69,"Manufactures of metals, nes"
303,5826,1.724977,9.0,58,"Artificial resins and plastic materials, and c..."
571,7373,1.723591,10.0,73,Metalworking machinery


In [ ]:
#top 10 least complex product
pci_bottom = pci_two.sort_values(by='rank').tail(10)
pci_bottom

,sitc,0,rank,two,Commodity description
77,0721,-2.251117,758.0,07,"Coffee, tea, cocoa, spices, and manufactures t..."
230,4244,-2.265757,759.0,42,Fixed vegetable oils and fats
122,2320,-2.271234,760.0,23,Crude rubber (including synthetic and reclaimed)
766,9710,-2.306943,761.0,97,"Gold, non-monetary (excluding gold ores and co..."
110,2221,-2.307036,762.0,22,Oil seeds and oleaginous fruit
196,2922,-2.450628,763.0,29,"Crude animal and vegetable materials, nes"
142,2631,-2.644727,764.0,26,Textile fibres (not wool tops) and their waste...
114,2225,-2.645427,765.0,22,Oil seeds and oleaginous fruit
209,3330,-2.971170,766.0,33,"Petroleum, petroleum products and related mate..."
188,2876,-3.199582,767.0,28,Metalliferous ores and metal scrap


In [64]:
d=1-distance
d

country,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,CAF,...,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE
productcode1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0011,0.055550,0.083455,0.016523,0.068567,0.164961,0.128281,0.043079,0.032411,0.068031,0.196470,0.118063,0.039150,0.021269,0.029828,0.046880,0.153581,0.472525,0.034533,0.048343,0.405718,0.073819,0.012307,0.034674,0.077175,0.416849,0.067934,0.045489,0.311485,0.079737,0.310878,0.073033,0.029132,0.089502,0.218568,0.132745,0.011224,0.037920,0.013508,0.039439,0.028011,...,0.352988,0.398094,0.337796,0.131776,0.033348,0.033948,0.161671,0.069676,0.015209,0.088661,0.324654,0.046842,0.101215,0.035859,0.027995,0.057183,0.048101,0.202330,0.414393,0.004514,0.170139,0.228755,0.185022,0.320918,0.044028,0.173257,0.365830,0.091768,0.064412,0.037628,0.033976,0.073732,0.207164,0.030251,0.012456,0.148617,0.072780,0.224795,0.112962,0.112165
0012,0.053113,0.138021,0.024517,0.058288,0.189913,0.133563,0.044450,0.031133,0.087631,0.219993,0.163655,0.042655,0.030632,0.028132,0.047721,0.178838,0.382813,0.048632,0.052335,0.361233,0.075765,0.015068,0.058466,0.097914,0.390140,0.076173,0.050286,0.273442,0.075811,0.282894,0.077750,0.031863,0.105271,0.200486,0.129609,0.013595,0.036243,0.015073,0.043579,0.036839,...,0.293453,0.330174,0.278367,0.133988,0.032935,0.035519,0.208178,0.065236,0.028496,0.095239,0.294894,0.069705,0.108116,0.055163,0.029000,0.060284,0.044242,0.228867,0.444569,0.002692,0.153359,0.286129,0.199973,0.285317,0.040885,0.158752,0.335318,0.126132,0.060073,0.041479,0.039692,0.069364,0.219887,0.036426,0.012021,0.146154,0.115551,0.243452,0.121290,0.135966
0013,0.050725,0.053445,0.011600,0.059916,0.105247,0.118241,0.046208,0.029756,0.052282,0.152133,0.083091,0.034098,0.018806,0.021786,0.036225,0.109007,0.442474,0.025185,0.030688,0.457763,0.049421,0.011254,0.019074,0.050323,0.327823,0.060308,0.035996,0.234194,0.070708,0.289269,0.055138,0.027734,0.056872,0.177644,0.110691,0.008826,0.027423,0.014108,0.024382,0.016889,...,0.336407,0.348155,0.339472,0.100350,0.028995,0.025497,0.117744,0.062276,0.006376,0.061673,0.338306,0.026875,0.085932,0.032076,0.022446,0.043779,0.035550,0.161335,0.368700,0.003608,0.220362,0.176684,0.141846,0.255717,0.042382,0.131749,0.361643,0.071876,0.056798,0.028903,0.022353,0.071484,0.178581,0.020411,0.010369,0.120157,0.044647,0.177995,0.072952,0.078607
0014,0.064820,0.071324,0.014611,0.067496,0.154593,0.134947,0.044362,0.035765,0.062515,0.181178,0.116458,0.040175,0.020398,0.030062,0.045167,0.116259,0.466305,0.033729,0.042764,0.440380,0.066689,0.013892,0.030534,0.081976,0.401815,0.072717,0.039834,0.273986,0.082441,0.306277,0.066937,0.030571,0.076287,0.203152,0.147992,0.011492,0.036918,0.013746,0.033905,0.022678,...,0.331867,0.369272,0.339562,0.143893,0.034699,0.032702,0.167099,0.068604,0.013263,0.084613,0.306424,0.040779,0.109478,0.038789,0.025470,0.058264,0.048616,0.208211,0.443947,0.004202,0.185389,0.219440,0.179031,0.275997,0.044858,0.147424,0.367631,0.088718,0.060772,0.036900,0.027832,0.070260,0.204378,0.025744,0.012479,0.138860,0.062242,0.212230,0.114995,0.104963
0015,0.065787,0.076539,0.014266,0.055313,0.108069,0.158248,0.051739,0.032206,0.055855,0.223640,0.094592,0.042336,0.026856,0.024171,0.036974,0.165409,0.414248,0.027128,0.035872,0.460912,0.051830,0.015783,0.025175,0.055710,0.299306,0.063060,0.040902,0.194713,0.061026,0.259988,0.066441,0.053440,0.078072,0.180263,0.143374,0.014875,0.025827,0.012943,0.031786,0.021947,...,0.243573,0.295687,0.301560,0.104585,0.030904,0.028854,0.132418,0.051670,0.015375,0.069790,0.267537,0.038809,0.074542,0.035659,0.022676,0.048497,0.035198,0.147214,0.312358,0.005277,0.204413,0.176896,0.149983,0.225155,0.048952,0.153226,0.450127,0.074988,0.052775,0.029577,0.025293,0.074161,0

In [70]:
oi= (d*rev_mcp)
oi= oi*pci
oi

,0,ABW,AFG,AGO,AIA,ALB,AND,ANS,ANT,ARE,ARG,ARM,ASM,ATA,ATF,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLM,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BVT,BWA,...,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TKL,TKM,TLS,TON,TTO,TUN,TUR,TUV,TWN,TZA,UGA,UKR,UMI,URY,USA,UZB,VAT,VCT,VEN,VGB,VNM,VUT,WLF,WSM,YEM,ZAF,ZMB,ZWE,sitc
0011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
